# Weekly Merge
Notebook to facilitate the weekly merge with Maddie's Data

In [1]:
import pandas as pd
import numpy as np

In [36]:
# Read in data
input_data = pd.read_csv("input_data.csv")
maddie = pd.read_excel("Week-by-week elimination table.ods", engine='odf')

In [57]:
# Tweak Maddie's show names
maddie = maddie.replace({"Bachelor":"The Bachelor"})

In [58]:
maddie.head()

,Show,Contestant,Week 1,Week 2,Week 3 Part 1,Week 3 Part 2,Week 4,Week 5
0,The Bachelor,Alayah,0,0,0,0,0,0
1,The Bachelor,Alexa,0,0,0,0,0,0
2,The Bachelor,Deandra,1,0,0,0,0,0
3,The Bachelor,Hannah Ann,1,1,1,1,1,1
4,The Bachelor,Jasmine,0,0,0,0,0,0


In [59]:
# Trim my data (I trust Maddie more)
# input_data.drop(columns=["Week 1", "Week 2", "Score"], inplace=True)
input_data = input_data[["Show", "Name", "file", "PickedBy", "Eliminated"]]

In [60]:
# Merge on show and contenstant
df = pd.merge(input_data, maddie, left_on=["Show", "Name"],
              right_on=['Show', "Contestant"], how='left')

In [61]:
df.Show.unique()

array(['Kids Baking Championship', 'Project Runway', 'Survivor',
       'The Bachelor', 'Top Chef'], dtype=object)

In [62]:
df.loc[df.Show == "The Bachelor"]

,Show,Name,file,PickedBy,Eliminated,Contestant,Week 1,Week 2,Week 3 Part 1,Week 3 Part 2,Week 4,Week 5
39,The Bachelor,Alayah,bach_alayah.png,Brock,1,Alayah,0,0,0,0,0.0,0.0
40,The Bachelor,Alexa,bach_alexa.png,Matt,1,Alexa,0,0,0,0,0.0,0.0
41,The Bachelor,Deandra,bach_deandra.png,Maddie,1,Deandra,1,0,0,0,0.0,0.0
42,The Bachelor,Hannah Ann,bach_hannahann.png,Martyna,1,Hannah Ann,1,1,1,1,1.0,1.0
43,The Bachelor,Jasmine,bach_jasmine.png,Jessica,1,Jasmine,0,0,0,0,0.0,0.0
44,The Bachelor,Kelley,bach_kelley.png,Maddie,1,Kelley,1,1,1,1,0.0,0.0
45,The Bachelor,Kelsey,bach_kelsey.png,Shelley,1,Kelsey,1,1,1,1,1.0,0.0
46,The Bachelor,Kiarra,bach_kiarra.png,Matt,1,Kiarra,1,0,0,0,0.0,0.0
47,The Bachelor,Lexi,bach_lexi.png,Alex G,1,Lexi,1,1,0,0,0.0,0.0
48,The Bachelor,Madison,bach_madison.png,Nate,1,Madison,1,1,1,1,1.0,1.0


In [63]:
# Fill X and NA with 0
df.fillna(0., inplace=True)
df.replace({"X":0.}, inplace=True)

In [64]:
# Change datatypes for weeks
for col in df.columns:
    if "Week" in col:
        df[col] = df[col].astype(float)

In [65]:
# Get Scores
week_cols = list(filter(lambda x: "Week" in x, list(df.columns)))
df['Score'] = df[week_cols].sum(axis=1)

In [66]:
# Handle eliminated
last_week = week_cols[-1]
df['Eliminated'] = np.where(df[last_week] == 0, 1, 0)

In [69]:
# Fix for Top Chef (hasn't started)
df['Eliminated'] = np.where(df.Show == "Top Chef", 0, df.Eliminated)

In [70]:
df.to_csv("input_data.csv")

In [71]:
## Save to JSON
s = df.to_json(orient='records')
f = open("data.js", 'w+')
f.write("var data = " + s + ';')
f.close()